In [1]:
import json
import os
import time
from datetime import datetime

from os.path import dirname, realpath
import sys
sys.path.append(dirname(dirname(realpath(os.getcwd()))))
import pickle
import torch
import multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP
from collections import Counter
from copy import deepcopy
import datetime
import tqdm
import numpy as np
import random
import pandas as pd
src_path2 = "G:\\FillmoreCancerData\\markhe\\VTERisk - Copy" 

from collections import Counter
sys.path.append("./src/")

from sklearn.metrics import confusion_matrix

import datetime

def parse_date(date_str):
    if date_str == 'NA' or date_str is None:
        return datetime.datetime(9999,1,1,0,0)
    else:
        if len(date_str) == 10:
            format_str = '%Y-%m-%d'
        else:
            raise Exception("Format for {} not recognized!".format(date_str))
    return datetime.datetime.strptime(date_str, format_str)
VTE_path = 'Y:\\Fillmore_Cancer\\cat\\Users\\data\\derived\\vte_ml\\vte_ml\\data_for_chunlei'

In [17]:
X_test = json.load(open(os.path.join(os.path.join(VTE_path, 'A2_CMP\\test2\\test.json')), 'r'))

In [18]:
X_dev = json.load(open(os.path.join( os.path.join(VTE_path, 'A2_CMP\\dev2\\dev.json')), 'r'))

In [19]:
X_train = json.load(open(os.path.join( os.path.join(VTE_path, 'A2_CMP\\train2\\train.json')), 'r'))

In [20]:
def sample_from_dict(d, sample=10):
    keys = random.sample(list(d), sample)
    values = [d[k] for k in keys]
    return dict(zip(keys, values))

In [4]:
#Y:\Fillmore_Cancer\cat\Users\data\derived\vte_ml\vte_ml\data_for_chunlei\A2_CMP\Resplit\Y2

In [5]:
dev_path = os.path.join(VTE_path, 'A2_CMP\Resplit\Y2\\dev\\dev.json')
test_path = os.path.join(VTE_path, 'A2_CMP\Resplit\Y2\\test\\test.json')
train_path = os.path.join(VTE_path, 'A2_CMP\Resplit\Y2\\train\\train.json')


In [21]:
Y_train = sample_from_dict(X_train, 5000)

In [22]:
Y_dev = sample_from_dict(X_dev, 2500)

In [23]:
Y_test = sample_from_dict(X_test, 2500)

In [24]:
with open(train_path, 'w') as fwd:
    print("Saving to {}\n".format(train_path))
    json.dump(Y_train, fwd, indent=None)

Saving to Y:\Fillmore_Cancer\cat\Users\data\derived\vte_ml\vte_ml\data_for_chunlei\A2_CMP\Resplit\Y2\train\train.json



In [25]:
with open(test_path, 'w') as fwd:
    print("Saving to {}\n".format(test_path))
    json.dump(Y_test, fwd, indent=None)

Saving to Y:\Fillmore_Cancer\cat\Users\data\derived\vte_ml\vte_ml\data_for_chunlei\A2_CMP\test2\test.json



In [26]:
with open(dev_path, 'w') as fwd:
    print("Saving to {}\n".format(dev_path))
    json.dump(Y_dev, fwd, indent=None)

Saving to Y:\Fillmore_Cancer\cat\Users\data\derived\vte_ml\vte_ml\data_for_chunlei\A2_CMP\Resplit\Y2\dev\dev.json



In [33]:
testDF = pd.read_csv(os.path.join(src_path2, 'Notebooks/Find/fixed_dx.csv'))


In [53]:
testDF

,Unnamed: 0,patient_id,outcome,dob,outcome_date,obs_time_end,index_date,dx_date,obs_time_start,diag_date,Id_diag_diff
0,1,1004880340,False,1921-09-04,2011-02-05,2011-02-05,2006-03-24,2008-06-02,2000-06-26,2006-02-06,46
1,2,1014382133,True,1948-03-12,2016-11-09,2018-12-13,2016-10-21,2021-01-29,2012-06-26,2016-09-07,44
2,3,1002850316,False,1925-08-08,2018-03-25,2018-03-25,2017-12-29,2006-08-21,1999-10-19,2017-11-27,32
3,4,1040528173,False,1944-12-21,2023-03-24,2023-03-24,2021-06-02,2021-06-02,2016-10-20,2021-04-20,43
4,5,1012154223,False,1913-08-10,2009-01-04,2009-01-04,2008-08-28,2015-06-15,2005-05-23,2008-08-26,2
...,...,...,...,...,...,...,...,...,...,...,...
123643,123644,1007436652,False,1929-12-15,2007-12-02,2007-12-02,2007-08-09,2010-12-13,2001-11-16,2007-08-03,6
123644,123645,1010685565,False,1941-03-19,2019-11-13,2019-11-13,2019-10-21,2018-11-26,2016-03-25,2019-06-03,140
123645,123646,1003074961,False,1947-11-19,2010-06-24,2010-06-24,2009-08-18,2016-06-27,1999-10-06,2009-07-02,47
123646,123647,1021189617,False,1947-03-07,2021-05-03,2021-05-03,2018-05-18,2018-12-11,2013-11-06,2018-03-12,67


In [34]:
VTE_path = 'Y:\\Fillmore_Cancer\\cat\\Users\\data\\derived\\vte_ml\\vte_ml\\data_for_chunlei'

In [57]:
m = testDF[["patient_id", "index_date","diag_date", "obs_time_end",  "outcome","outcome_date" ]]

m_VE = m[ m['outcome'] ]
m_VE1 = m_VE [   (pd.to_datetime( m_VE["outcome_date"] ) -pd.to_datetime( m_VE["index_date"] )) < datetime.timedelta(730)]
m_VE0 = m_VE [   (pd.to_datetime( m_VE["outcome_date"] ) -pd.to_datetime( m_VE["index_date"] )) >= datetime.timedelta(730)]
m_VE0["outcome"] = m_VE0["outcome"].replace(True, False)  
m0= m[ ~ m['outcome'] ]
m1 = pd.concat([m_VE1, m_VE0, m0])
m1['PatICN'] =  m1.patient_id.astype(str)

m_VE0['model_date'] = pd.to_datetime(m_VE0['index_date']) + datetime.timedelta(730)
m_VE0['PatICN'] =  m_VE0.patient_id.astype(str)

C:\Users\VHANYNHeT\AppData\Local\Temp\2\ipykernel_16324\1629420586.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m_VE0["outcome"] = m_VE0["outcome"].replace(True, False)
C:\Users\VHANYNHeT\AppData\Local\Temp\2\ipykernel_16324\1629420586.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m_VE0['model_date'] = pd.to_datetime(m_VE0['index_date']) + datetime.timedelta(730)
C:\Users\VHANYNHeT\AppData\Local\Temp\2\ipykernel_16324\1629420586.py:12: SettingWithCopyWarning: 
A value is trying to be set on a 

In [58]:
m_VE0

,patient_id,index_date,diag_date,obs_time_end,outcome,outcome_date,model_date,PatICN
42,1003403917,2008-08-08,2008-05-20,2023-03-31,False,2014-02-27,2010-08-08,1003403917
81,1005750539,2008-08-16,2008-02-11,2015-04-21,False,2015-01-12,2010-08-16,1005750539
131,1003885905,2008-12-15,2007-12-28,2023-05-16,False,2016-04-05,2010-12-15,1003885905
159,1008201009,2010-04-13,2010-03-04,2015-04-10,False,2014-03-03,2012-04-12,1008201009
191,1005967537,2020-04-09,2020-01-13,2023-05-03,False,2023-03-21,2022-04-09,1005967537
...,...,...,...,...,...,...,...,...
123527,1018836377,2016-12-27,2016-11-02,2023-05-23,False,2020-02-04,2018-12-27,1018836377
123542,1004145178,2017-09-08,2017-07-12,2020-08-27,False,2019-09-27,2019-09-08,1004145178
123565,1012258117,2015-03-12,2014-11-03,2023-05-01,False,2017-06-12,2017-03-11,1012258117
123570,1003443658,2010-03-10,2010-02-11,2018-01-26,False,2015-07-15,2012-03-09,1003443658


In [59]:
p = '1004690584'

In [69]:
Y = X

In [70]:

for p in Y.keys():
    
    Y[p]['dxdate'] =  m1 [ m1 ['PatICN']==p ]['diag_date'].values[0]

    if sum( p  ==  m_VE0.PatICN ) ==1 :
        m_p = m_VE0[  p  ==  m_VE0.PatICN ] 
        Yp = Y[p]['events']

        m_date = m_p['model_date'].astype('datetime64[ns]')
        Yp_f = filter(lambda x: parse_date(x['admdate']) < m_date.ravel()[0] , Yp)
        Yp_new = list( Yp_f)
        Y[p]['events']  = Yp_new 
    

In [71]:
os.mkdir(os.path.join(VTE_path, 'A2_CMP\\dev2'))

In [72]:
adjdev = os.path.join(VTE_path, 'A2_CMP\\dev2\\dev.json')


In [75]:
with open(adjdev, 'w') as fwd:
    print("Saving to {}\n".format(adjdev))
    json.dump(Y, fwd, indent=None)

Saving to Y:\Fillmore_Cancer\cat\Users\data\derived\vte_ml\vte_ml\data_for_chunlei\A2_CMP\dev2\dev.json



In [74]:
test_path

'Y:\\Fillmore_Cancer\\cat\\Users\\data\\derived\\vte_ml\\vte_ml\\data_for_chunlei\\A2_CMP\\dev\\dev.json'